In [4]:
import os
from glob import glob
from tqdm.notebook import tqdm
from coreLib.utils import *
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
# data paths
data_path="/media/ansary/DriveData/Work/LEAF/archive/"
train_path=os.path.join(data_path,'train')
eval_path=os.path.join(data_path,'validation')
# save paths
save_path="/media/ansary/DriveData/Work/LEAF/"
save_path=create_dir(save_path,'data')
train_save=create_dir(save_path,'train')
test_save=create_dir(save_path,'test')

# image aug
ia.seed(1)
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order


In [11]:

data_dim=256
n_aug=20
train_paths=[_path for _path in glob(os.path.join(train_path,"*/*.jpg"))]
for idx,img_path in tqdm(enumerate(train_paths),total=len(train_paths)):
    img=cv2.imread(img_path)
    img=cv2.resize(img,(data_dim,data_dim))
    label=os.path.basename(os.path.dirname(img_path))
    label_dir=create_dir(train_save,label)
    cv2.imwrite(os.path.join(label_dir,f"{idx}.png"),img)
    # aug
    images=[img for _ in range(n_aug)]
    images_aug = seq(images=images)
    for iidx,img_aug in enumerate(images_aug):
        cv2.imwrite(os.path.join(label_dir,f"{idx}_{iidx}.png"),img_aug)
    

  0%|          | 0/1600 [00:00<?, ?it/s]

In [10]:
test_paths=[_path for _path in glob(os.path.join(eval_path,"*/*.jpg"))]
for idx,img_path in tqdm(enumerate(test_paths),total=len(test_paths)):
    img=cv2.imread(img_path)
    img=cv2.resize(img,(data_dim,data_dim))
    label=os.path.basename(os.path.dirname(img_path))
    label_dir=create_dir(test_save,label)
    cv2.imwrite(os.path.join(label_dir,f"{idx}.png"),img)


  0%|          | 0/492 [00:00<?, ?it/s]